Imports

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')


SyntaxError: invalid syntax (2303010437.py, line 3)

In [1]:
#Imports
import pandas as pd
import re
import numpy as np
from collections import Counter
import nltk
nltk.download('punkt')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amandadotkom/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loading dataset

In [12]:
# Read file
data = pd.read_csv('/content/drive/MyDrive/Bachelor Project 2023/Data/selection_FIC_Juv.txt', sep='delimiter', header=None, encoding = 'utf-8')
data.set_axis(['Text'], axis='columns', inplace=True)
data.head()

<ipython-input-12-e9af65ee0e3b>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/content/drive/MyDrive/Bachelor Project 2023/Data/selection_FIC_Juv.txt', sep='delimiter', header=None, encoding = 'utf-8')
<ipython-input-12-e9af65ee0e3b>:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  data.set_axis(['Text'], axis='columns', inplace=True)


,Text
0,@@4121408 Mary Jane woke up excited . She had ...
1,@@4121411 Earth has become too polluted to liv...
2,"@@4121413 "" Perfect storm ? Lefs show them ! ""..."
3,"@@4121415 BE CAREFUL WHAT YOU WISH FOR , IT MI..."
4,"@@4121428 When the Mee family moves , they get..."


In [13]:
# Store all the fictional document IDs in a list
docs = data['Text']
ids = []
idx = 0
for ID in docs:
  word = docs[idx].split()
  ids.append(word[0])
  # print(ids[idx])
  idx += 1


In [14]:
# Make dataframe that has the ID on one column, and each sentence from that ID on the other columns

id_and_text = pd.DataFrame(list(zip(ids, docs)))
id_and_text.set_axis(['ID', 'Text'], axis='columns', inplace=True)


id_and_text.head()

<ipython-input-14-26c6984a8b5c>:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  id_and_text.set_axis(['ID', 'Text'], axis='columns', inplace=True)


,ID,Text
0,@@4121408,@@4121408 Mary Jane woke up excited . She had ...
1,@@4121411,@@4121411 Earth has become too polluted to liv...
2,@@4121413,"@@4121413 "" Perfect storm ? Lefs show them ! ""..."
3,@@4121415,"@@4121415 BE CAREFUL WHAT YOU WISH FOR , IT MI..."
4,@@4121428,"@@4121428 When the Mee family moves , they get..."


###Data cleaning and preprocessing

In [15]:
# Based on https://www.kaggle.com/code/boldy717/textual-data-exploration-with-n-grams
def clean(documents):
    cleaned_text = []
    for document in documents:
        s = re.sub(r'<.*?>', '', document) # Removes the <p>
        s = s.replace("n't", " not")
        s = s.replace("'s", " is")
        s = re.sub(r'[^a-zA-Z\s.]', '', s) # non alphanumeric or whitespaces, keeps periods so sentence structure remains
        s = re.sub('\s+',' ', s)
        s = str(s).lower()
        tokens = [token for token in s.split(" ") if token != ""]
        # print(tokens)
        book = ' '.join(tokens)
        cleaned_text.append(book)
    return(cleaned_text)


In [16]:
id_and_text['Text'] = clean(docs)

In [17]:
id_and_text.head()

,ID,Text
0,@@4121408,mary jane woke up excited . she had barely sle...
1,@@4121411,earth has become too polluted to live on . to ...
2,@@4121413,perfect storm lefs show them said the uuind to...
3,@@4121415,be careful what you wish for it might come tru...
4,@@4121428,when the mee family moves they get a big backy...


In [18]:
# for form {ID : all sentences from that ID}
def split_sentences(df):
  for idx, item in enumerate(df['Text']):
    df['Text'][idx] = df['Text'][idx].split('.')



In [19]:
split_sentences(id_and_text)

In [20]:
id_and_text.head()

,ID,Text
0,@@4121408,"[mary jane woke up excited , she had barely s..."
1,@@4121411,"[earth has become too polluted to live on , t..."
2,@@4121413,[perfect storm lefs show them said the uuind t...
3,@@4121415,[be careful what you wish for it might come tr...
4,@@4121428,[when the mee family moves they get a big back...


In [21]:
id_and_text['Text'][0][0]

'mary jane woke up excited '

###Saving/storing cleaned and preprocessed data to csv

In [22]:
from pathlib import Path
filepath = Path('DataFrame/IdAndText.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
id_and_text.to_csv(filepath)

In [23]:
import os
# os.makedirs('DataFrame', exist_ok=True)
id_and_text.to_csv('/content/drive/MyDrive/Bachelor Project 2023/Data/IdAndText.csv')

###Loading previously preprocessed data

In [24]:
import ast
# Reading the csv dataframe, using function below so that the 'Text' column will be read in as a list instead of a string
# https://saturncloud.io/blog/how-to-read-csv-containing-a-list-in-pandas/#:~:text=Reading%20CSV%20Files%20Containing%20Lists,list%20to%20an%20actual%20list.
# Define function to convert string to list
def str_to_list(s):
    return ast.literal_eval(s)

data = pd.read_csv('/content/drive/MyDrive/Bachelor Project 2023/Data/IdAndText.csv', encoding = 'utf-8', index_col=0, converters={'Text': str_to_list})

In [28]:
data.head()

,ID,Text
0,@@4121408,"[mary jane woke up excited , she had barely s..."
1,@@4121411,"[earth has become too polluted to live on , t..."
2,@@4121413,[perfect storm lefs show them said the uuind t...
3,@@4121415,[be careful what you wish for it might come tr...
4,@@4121428,[when the mee family moves they get a big back...


###Lemmatization

In [26]:
# lemmatizing
# use lemmas provided by the corpus

###Finding N-Grams

In [29]:
def get_ngrams(li, n):
  #takes in a list of sentences and n for n-gram
  # iterates thru list
  n_grams = []
  for idx, sentence in enumerate(li):
    words = sentence.split()
    if len(words) >= n: # skip sentences that are shorter than the n-gram
      ng = ngrams(words, n) # gets n grams of the sentence, creates tuples of those n grams
      ng = joined_ngrams = [' '.join(gram) for gram in ng] # convert the tuples in the ngrams zip to a string
      n_grams.append([item for item in ng]) # add the n gram strings to the list

  return n_grams

In [30]:
def store_ngrams(data, n):
  new_df = data.copy()
  for idx in range(len(data['Text'])):
    new_df['Text'][idx] = get_ngrams(data['Text'][idx], n)

  return new_df

In [31]:
bigrams = store_ngrams(data, 2)
trigrams= store_ngrams(data, 3)
fourgrams = store_ngrams(data, 4)
fivegrams = store_ngrams(data, 5)
sixgrams = store_ngrams(data, 6)

In [32]:
bigrams.head()

,ID,Text
0,@@4121408,"[[mary jane, jane woke, woke up, up excited], ..."
1,@@4121411,"[[earth has, has become, become too, too pollu..."
2,@@4121413,"[[perfect storm, storm lefs, lefs show, show t..."
3,@@4121415,"[[be careful, careful what, what you, you wish..."
4,@@4121428,"[[when the, the mee, mee family, family moves,..."


###Get most frequent n-grams

In [ ]:
bigrams['Text'][0]

In [34]:
lis = ['aba', 'aba', 'aba', 'me', 'me', 'sa']

occurence_count = Counter(lis)
print(occurence_count.most_common(1)[0][1])

print(occurence_count)

3
Counter({'aba': 3, 'me': 2, 'sa': 1})


In [35]:
# Put n-grams to one list
def make_list(df):
  listy = []
  for li in df['Text'][0]:
    for item in li:
      listy.append(item)

  return listy

In [37]:
# Use Counter function
# https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/

def most_frequent(li, baseline, min_occur):
    top_list = {}
    occurence_count = Counter(li) # dictionary consisting of word and its count

    # most_common() gets (baseline) number of most frequent items
    for word, freq in occurence_count.most_common(baseline):
      if freq > min_occur: # at least occurs more than (min)
        top_list[word] = freq

    return top_list

In [38]:
bigrams_list = make_list(bigrams)
trigrams_list = make_list(trigrams)
fourgrams_list = make_list(fourgrams)
fivegrams_list = make_list(fivegrams)
sixgrams_list = make_list(sixgrams)

In [39]:
top = most_frequent(trigrams_list, 100, 1)
print(top)

{'ice pop sticks': 4, 'the ice pop': 3, 'mary jane said': 3, 'tissue paper and': 3, 'the night before': 2, 'into the kitchen': 2, 'the kitchen table': 2, 'pop sticks and': 2, 'mom mary jane': 2, 'i was going': 2, 'was going to': 2, 'i did not': 2, 'it was not': 2, 'on the floor': 2, 'the living room': 2, 'is funny josh': 2, 'the colored tissue': 2, 'colored tissue paper': 2, 'that is funny': 2}
